THIS NOTEBOOK IS OUT OF DATE. Please see about_dicom2bids in the lcnimodules repo for correct usage of that module. If you read the original 4.BIDS notebook, it ended with conversion. We didn't get to that last week, so I've moved it here. Some things have changed (lucky you, taking a class using modules still in development!), so make sure you use the stuff in THIS notebook.

# An interlude: python's pathlib module
Just when I get used to one way of doing things (os.path functions), I find out there's a new way. Pythod versions 3.5 and higher introduce the pathlib module, which has significant overlap with os.path. It also is going answer some questions I've gotten about opening files in JupyterLab that aren't under your home directory

In [1]:
import pathlib

If you define this variable and the others in this section, it will be easier to follow along.

In [2]:
myhomedir = pathlib.Path.home()

In [3]:
myhomedir

PosixPath('/home/jolinda')

There are ways of seeing your groups/getting your pirg in python, but the easiest is to just use the groups command in bash.

In [4]:
!groups

talapas lcni


Your pirg should be listed there.

In [5]:
# set this variable to your own pirg
mypirg = 'lcni'

You should have a directory with the same name as your pirg in your home directory. We can check that like this.

In [6]:
my_pirgdir = myhomedir.joinpath(mypirg)

In [7]:
my_pirgdir

PosixPath('/home/jolinda/lcni')

In [8]:
my_pirgdir.exists()

True

In [9]:
my_pirgdir.is_symlink()

True

In [10]:
# see the full path
my_pirgdir.resolve()

PosixPath('/gpfs/projects/lcni/jolinda')

my_pirgdir is a symbolic link -- the real path is under projects/pirg, but because there is a link under my home directory I can access it from the left panel in Jupyter lab. Let's create a directory under it for our output today.

In [11]:
class5_output = my_pirgdir.joinpath('class5out')

In [12]:
# it doesn't actually exist yet
class5_output.exists()

False

In [13]:
# so let's make it
class5_output.mkdir()

In [14]:
class5_output.exists()

True

We can use pathlib to create symbolic links -- for example, to my shared TalapasClass folder. You might find this useful.

In [22]:
pathlib.Path('TalapasClass').symlink_to('/projects/lcni/jolinda/shared/TalapasClass')

Finally, a reminder about the system path (where python looks for modules). It's in the sys.path variable:

In [15]:
import sys
sys.path

['/gpfs/projects/lcni/jolinda/shared/TalapasClass',
 '',
 '/projects/lcni/shared/python3/site-packages',
 '/projects/lcni/jolinda/shared/site-packages',
 '/packages/miniconda/20190102/envs/anaconda-tensorflow-cpu/lib/python36.zip',
 '/packages/miniconda/20190102/envs/anaconda-tensorflow-cpu/lib/python3.6',
 '/packages/miniconda/20190102/envs/anaconda-tensorflow-cpu/lib/python3.6/lib-dynload',
 '/home/jolinda/.local/lib/python3.6/site-packages',
 '/home/jolinda/.local/lib/python3.6/site-packages/pyperclip-1.6.0-py3.6.egg',
 '/packages/miniconda/20190102/envs/anaconda-tensorflow-cpu/lib/python3.6/site-packages',
 '/packages/miniconda/20190102/envs/anaconda-tensorflow-cpu/lib/python3.6/site-packages/IPython/extensions',
 '/gpfs/home/jolinda/.ipython']

To follow along today you'll need /projects/lcni/jolinda/shared/site-packages in your path (or another path to the lcnimodules if you've gotten them from github). If you haven't added the directory containing those modules to your path you'll need to do so now:

In [16]:
if '/projects/lcni/jolinda/shared/site-packages' not in sys.path:
    sys.path.append('/projects/lcni/jolinda/shared/site-packages')

To avoid doing this all the time, set the path in your .bashrc and .bashrc_ondemand files in your home directory. I have a sample file in my shared directory with this and some other useful environment variables.

In [17]:
!cat /projects/lcni/jolinda/shared/sample.bashrc

# .bashrc and .bashrc-ondemand

export PYTHONPATH=$PYTHONPATH:/projects/lcni/jolinda/shared/site-packages
export SLURM_ACCOUNT=<YOUR PIRG HERE>
export SBATCH_ACCOUNT=<YOUR PIRG HERE>


# Review: BIDS directory structure

In [18]:
!tree /projects/lcni/jolinda/shared/bids_example -L 1 # top level only

/projects/lcni/jolinda/shared/bids_example
|-- dataset_description.json
|-- participants.json
|-- participants.tsv
|-- sub-TC001
`-- sub-TC002

2 directories, 3 files


One thing that I've added since last week: the participants.json file. What's that about? First, look at the participants.tsv file.

In [19]:
with open ('/projects/lcni/jolinda/shared/bids_example/participants.tsv') as f:
    print(f.read())

participant_id	age	sex
sub-TC002	18	O
sub-TC001	12	O



You can have as many columns as you like, but first column must be participant_id. Anything beyond age, sex, or group must be defined in the participants.json file. We don't need one with this data set, but I made one anyway to show you what it should look like.

In [20]:
# this is how to "pretty print" a json file in python
# Load the file (json.load), dump it to a string (json.dumps), print the string
import json
with open ('/projects/lcni/jolinda/shared/bids_example/participants.json') as f:
    j = json.load(f)
print(json.dumps(j, indent = 4))

{
    "age": {
        "Description": "age of participant",
        "Units": "years"
    },
    "sex": {
        "Description": "sex of participant",
        "Levels": {
            "M": "male",
            "F": "female",
            "O": "other"
        }
    }
}


In [21]:
with open ('/projects/lcni/jolinda/shared/bids_example/dataset_description.json') as f:
    j = json.load(f)
print(json.dumps(j, indent = 4))

{
    "Name": "TC",
    "BIDSVersion": "1.3.0",
    "Authors": [
        "Jolinda Smith"
    ],
    "Acknowledgements": "BIDS conversion was performed using dcm2niix. Thanks to Jolinda Smith at LCNI at the University of Oregon for additional scripting of BIDS conversion.",
    "ReferencesAndLinks": [
        "Li X, Morgan PS, Ashburner J, Smith J, Rorden C (2016) The first step for neuroimaging data analysis: DICOM to NIfTI conversion. J Neurosci Methods. 264:47-56. doi: 10.1016/j.jneumeth.2016.03.001."
    ]
}


There's more you can put in the dataset description. The only things required are Name and BIDSVersion.

Below the subject level is the optional session level. Below that, is the 'file type' level, with a directory for each of the standard file types (anat, func, dwi, etc). The lowest level contains nifti files and matching json "sidecar" files. 

In [22]:
!tree /projects/lcni/jolinda/shared/bids_example/sub-TC002/

/projects/lcni/jolinda/shared/bids_example/sub-TC002/
`-- ses-pre
    |-- anat
    |   |-- sub-TC002_ses-pre_acq-mprage_run-2_T1w.json
    |   `-- sub-TC002_ses-pre_acq-mprage_run-2_T1w.nii.gz
    `-- func
        |-- sub-TC002_ses-pre_task-SVC_run-3_bold.json
        `-- sub-TC002_ses-pre_task-SVC_run-3_bold.nii.gz

3 directories, 4 files


The file names all have the format: `key-{value}_key-{value}_key-{value}_suffix.extension`. The key-{value} pairs are "entities" in bids-speak. Some are required, some are not (see https://bids-specification.readthedocs.io/en/stable/99-appendices/04-entity-table.html). 

# Conversion
First, let's store the path to a directory of files we want to convert in a variable. Feel free to use a folder of files you have access to.

In [23]:
dcmdir = '/projects/lcni/dcm/TalapasClass/'

Remember our lcni standard structure? pirg -- PI -- project -- subject -- series -- files

In [24]:
!tree {dcmdir} -d

/projects/lcni/dcm/TalapasClass/
`-- Smith
    `-- TC
        |-- TC001_20180620_110428
        |   |-- Series_10_bold_DIFTIM_1.7mm
        |   |-- Series_11_bold_DIFTIM_1.7mm
        |   |-- Series_1_AAHead_Scout_32ch-head-coil
        |   |-- Series_2_AAHead_Scout_32ch-head-coil_MPR_sag
        |   |-- Series_3_AAHead_Scout_32ch-head-coil_MPR_cor
        |   |-- Series_4_AAHead_Scout_32ch-head-coil_MPR_tra
        |   |-- Series_5_mprage_p2
        |   |-- Series_6_bold_DIFTIM_1.7mm
        |   |-- Series_7_bold_DIFTIM_1.7mm
        |   |-- Series_8_se_epi_1.7mm_ap
        |   `-- Series_9_se_epi_1.7mm_pa
        `-- TC002_20180916_090647
            |-- Series_1_localizer
            |-- Series_2_mprage_p2
            `-- Series_3_SVC_1_bold_mb3_g2_2mm_te25

18 directories


Our first job: to create a mapping of series folder names to bids-compliant output file names.  
It's helpful to collect all the series names from our dicom directory. dicom2bids has a function to do just that.

In [25]:
import dicom2bids

In [26]:
dicom2bids.GetSeriesNames(dcmdir)

{'AAHead_Scout_32ch-head-coil',
 'AAHead_Scout_32ch-head-coil_MPR_cor',
 'AAHead_Scout_32ch-head-coil_MPR_sag',
 'AAHead_Scout_32ch-head-coil_MPR_tra',
 'SVC_1_bold_mb3_g2_2mm_te25',
 'bold_DIFTIM_1.7mm',
 'localizer',
 'mprage_p2',
 'se_epi_1.7mm_ap',
 'se_epi_1.7mm_pa'}

For each of these series, we need to: 
1. Decide whether to convert it
2. Figure out the file type (anat, func, dwi, fmap)
3. Define the 'entity-chain' for the output file names  

If you cannot do this much, take some time to learn what the different series in your experiment ACTUALLY ARE.  
The scouts and localizers are usually not converted. That leaves two bold series, two field map series, and an anatomical. Let's start with the anatomical. The only required entity is subject, but we can't define that yet. The suffix is always required, and for file type anat can be:

In [27]:
dicom2bids.suffixes['anat']

['T1w',
 'T2w',
 'FLAIR',
 'T1rho',
 'T1map',
 'T2map',
 'T2star',
 'FLASH',
 'PD',
 'PDmap',
 'PDT2',
 'inplaneT1',
 'inplaneT2',
 'angio',
 'defacemask']

We all remember that mprages collected with our functional scans are usually T1 weighted (right?), so we choose 'T1w'. Our 'mapping' then looks like this:

In [28]:
bd = dicom2bids.bids_dict() # create a bids dictionary

In [29]:
bd.add('mprage_p2', datatype = 'anat', suffix = 'T1w') 

In [30]:
print(bd)

mprage_p2: sub-{}_T1w



If we decide we want to include the optional 'acq' entity, we can do that by defining our entity chain

In [31]:
bd.add('mprage_p2', datatype = 'anat', suffix = 'T1w', chain = {'acq': 'mprage'}) 

NEW: since I first made this notebook I updated dicom2bids. You can now write:
```
bd.add('mprage_p2', datatype = 'anat', suffix = 'T1w', acq = 'mprage') 
```
However, you'll still need to use the 'chain' language for any bids entities that are also python keywords (such as dir)

In [32]:
print(bd)

mprage_p2: sub-{}_acq-mprage_T1w



dicom2bids keeps track of what order the entities need to be in and (to some extent) whether they are valid. For example, if we try an uppercase W:

In [33]:
bd.add('mprage_p2', datatype = 'anat', suffix = 'T1W', chain = {'acq': 'mprage'}) 

ValueError: Unknown suffix T1W for data type anat
Allowed suffixes are ['T1w', 'T2w', 'FLAIR', 'T1rho', 'T1map', 'T2map', 'T2star', 'FLASH', 'PD', 'PDmap', 'PDT2', 'inplaneT1', 'inplaneT2', 'angio', 'defacemask']

It IS possible to use a non-standard suffix. To do that we just need to use the nonstandard = True flag, and the dictionary will still accept it.

In [34]:
bd.add('t3_weighted_sequence', datatype = 'anat', suffix = 'T3w', nonstandard = True) 

In [35]:
print(bd)

mprage_p2: sub-{}_acq-mprage_T1w
t3_weighted_sequence: sub-{}_T3w



What about sessions? Suppose these scans are all from the "pre" session. Then we just need to add the 'ses' label to our chains. dicom2bids doesn't check for this (yet), but if one scan has the ses label then they all should have it.

In [36]:
bd.add('mprage_p2', datatype = 'anat', suffix = 'T1w', chain = {'acq': 'mprage', 'ses':'pre'}) 
bd.add('t3_weighted_sequence', datatype = 'anat', suffix = 'T3w', nonstandard = True, chain = {'ses':'pre'}) 

Next up: the bold sequences. According to the entity table, the 'task' entity is required.

In [37]:
bd.add('SVC_1_bold_mb3_g2_2mm_te25', datatype = 'func', suffix = 'bold', chain = {'task':'SVC', 'ses':'pre'}) 
bd.add('bold_DIFTIM_1.7mm', datatype = 'func', suffix = 'bold',  chain = {'task':'DIFTIM', 'ses':'pre'})

Finally, the field maps. 

In [38]:
bd.add('se_epi_1.7mm_ap', datatype = 'fmap', suffix = 'epi', chain = {'dir':'ap', 'ses':'pre'}) 
bd.add('se_epi_1.7mm_pa', datatype = 'fmap', suffix = 'epi', chain = {'dir':'pa', 'ses':'pre'}) 

In [39]:
print(bd)

mprage_p2: sub-{}_ses-pre_acq-mprage_T1w
t3_weighted_sequence: sub-{}_ses-pre_T3w
SVC_1_bold_mb3_g2_2mm_te25: sub-{}_ses-pre_task-SVC_bold
bold_DIFTIM_1.7mm: sub-{}_ses-pre_task-DIFTIM_bold
se_epi_1.7mm_ap: sub-{}_ses-pre_dir-ap_epi
se_epi_1.7mm_pa: sub-{}_ses-pre_dir-pa_epi



You may have noticed that there's a place for the subject name, but not for "run". Don't worry, it'll get filled in later. (If you define it in your chain now, it'll get overwritten later, so don't do that.)

So we have our input directory and our bids dictionary, we just need our output directory. Don't use the one I'm about to use -- you don't have rights to write to it. Pick something in your own pirg's directory. This does not have to be a directory that exists, but you must be able to create it.

In [40]:
bidsdir = class5_output.joinpath('bids')

Everything is in place, we just need to call the convert function:

In [44]:
dicom2bids.Convert(dcmdir, bidsdir, bids_dict=bd, account = mypirg)

Submitted batch job 11618708

Submitted batch job 11618709



dicom2bids automatically submits one job per subject to the cluster. We'll talk more about that next week. The way to check a running job is to use sacct:

In [46]:
!sacct -j 11618708

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11618708        convert      short       lcni          1    RUNNING      0:0 
11618708.ba+      batch                  lcni          1    RUNNING      0:0 
11618708.ex+     extern                  lcni          1    RUNNING      0:0 


In [47]:
!sacct -j 11618709

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11618709        convert      short       lcni          1  COMPLETED      0:0 
11618709.ba+      batch                  lcni          1  COMPLETED      0:0 
11618709.ex+     extern                  lcni          1  COMPLETED      0:0 


Each job will creat a slurm-{jobnumber}.out file.

In [48]:
cat slurm-11618709.out

Chris Rorden's dcm2niiX version v1.0.20200331  (JP2:OpenJPEG) (JP-LS:CharLS) GCC5.5.0 (64-bit Linux)
Found 176 DICOM file(s)
Convert 176 DICOM as /home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/anat/sub-TC002_ses-pre_acq-mprage_run-02_T1w (256x256x176x1)
Compress: "/bin/pigz" -b 960 -n -f -6 "/home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/anat/sub-TC002_ses-pre_acq-mprage_run-02_T1w.nii"
Conversion required 2.548653 seconds (0.340000 for core code).
Chris Rorden's dcm2niiX version v1.0.20200331  (JP2:OpenJPEG) (JP-LS:CharLS) GCC5.5.0 (64-bit Linux)
Found 180 DICOM file(s)
Convert 180 DICOM as /home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/func/sub-TC002_ses-pre_task-SVC_run-03_bold (104x104x72x180)
Compress: "/bin/pigz" -b 960 -n -f -6 "/home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/func/sub-TC002_ses-pre_task-SVC_run-03_bold.nii"
Conversion required 16.228609 seconds (0.480000 for core code).


This is just whatever would print to stdout/stderr on the command line. Once our jobs are finished we can see the output with the tree command in bash.

In [49]:
!tree {bidsdir}

/home/jolinda/lcni/class5out/bids
|-- dataset_description.json
|-- participants.json
|-- participants.tsv
|-- sub-TC001
|   `-- ses-pre
|       |-- anat
|       |   |-- sub-TC001_ses-pre_acq-mprage_run-05_T1w.json
|       |   `-- sub-TC001_ses-pre_acq-mprage_run-05_T1w.nii.gz
|       |-- fmap
|       |   |-- sub-TC001_ses-pre_dir-pa_run-09_epi.json
|       |   `-- sub-TC001_ses-pre_dir-pa_run-09_epi.nii.gz
|       `-- func
|           |-- sub-TC001_ses-pre_task-DIFTIM_run-11_bold.json
|           |-- sub-TC001_ses-pre_task-DIFTIM_run-11_bold.nii
|           `-- sub-TC001_ses-pre_task-DIFTIM_run-11_bold.nii.gz
`-- sub-TC002
    `-- ses-pre
        |-- anat
        |   |-- sub-TC002_ses-pre_acq-mprage_run-02_T1w.json
        |   `-- sub-TC002_ses-pre_acq-mprage_run-02_T1w.nii.gz
        `-- func
            |-- sub-TC002_ses-pre_task-SVC_run-03_bold.json
            `-- sub-TC002_ses-pre_task-SVC_run-03_bold.nii.gz

9 directories, 14 files


About those sidecar .json files. There's something annoying in there.

In [50]:
import os
with open(os.path.join(bidsdir, 'sub-TC002', 'ses-pre', 'anat', 'sub-TC002_ses-pre_acq-mprage_run-02_T1w.json')) as f:
    j = json.load(f)
print(json.dumps(j, indent = 4))

{
    "Modality": "MR",
    "MagneticFieldStrength": 3,
    "ImagingFrequency": 123.22,
    "Manufacturer": "Siemens",
    "ManufacturersModelName": "Skyra",
    "InstitutionName": "Lewis_Building",
    "InstitutionalDepartmentName": "Department",
    "InstitutionAddress": "Franklin_Blvd_1440_Eugene_District_US_97403",
    "DeviceSerialNumber": "45348",
    "StationName": "AWP45348",
    "SeriesInstanceUID": "1.3.12.2.1107.5.2.19.45348.2018091609104434571015476.0.0.0",
    "StudyInstanceUID": "1.3.12.2.1107.5.2.19.45348.30000018091316524195700000031",
    "StudyID": "1",
    "PatientName": "TC002",
    "PatientID": "18.09.16-09:05:52-DST-1.3.12.2.1107.5.2.19.45348",
    "PatientBirthDate": "2000-01-01",
    "PatientSex": "O",
    "PatientWeight": 68.0389,
    "BodyPartExamined": "BRAIN",
    "PatientPosition": "HFS",
    "ProcedureStepDescription": "dsnlab_Pfeifer_TAG",
    "SoftwareVersions": "syngo_MR_E11",
    "MRAcquisitionType": "3D",
    "SeriesDescription": "mprage_p2",
    "Pro

The institution information is all messed up. I've fixed this on the scanner, so future files should come out okay, but if you're converting files that actually exist NOW then this will be wrong for anything from LCNI. Luckily I have a fix for this -- you can define a correction dictionary and include that in your call to Convert with the parameter "json_mod"

In [51]:
corrections = {'InstitutionName':'University of Oregon', 'InstitutionalDepartmentName':'LCNI', 'InstitutionAddress':'Franklin_Blvd_1440_Eugene_Oregon_US_97403'}

It's even easier than that for us. This is going to come up a lot so I've put those in the dicom2bids module as dicom2bids.lcni_corrections.

In [52]:
dicom2bids.lcni_corrections

{'InstitutionName': 'University of Oregon',
 'InstitutionalDepartmentName': 'LCNI',
 'InstitutionAddress': 'Franklin_Blvd_1440_Eugene_Oregon_US_97403'}

If you want to use other parameters with dcm2niix you can pass those too -- for example, only write the json sidecars, not the nifti files (-b o). You can also opt to just run the commands WITHOUT submitting a slurm job. We'll do that this time.

In [53]:
dicom2bids.Convert(dcmdir, bidsdir, bids_dict=bd, json_mod = dicom2bids.lcni_corrections, dcm2niix_flags='-b o', slurm = False)

In [54]:
import os
with open(os.path.join(bidsdir, 'sub-TC002', 'ses-pre', 'anat', 'sub-TC002_ses-pre_acq-mprage_run-02_T1w.json')) as f:
    j = json.load(f)
print(json.dumps(j, indent = 4))

{
    "Modality": "MR",
    "MagneticFieldStrength": 3,
    "ImagingFrequency": 123.22,
    "Manufacturer": "Siemens",
    "ManufacturersModelName": "Skyra",
    "InstitutionName": "University of Oregon",
    "InstitutionalDepartmentName": "LCNI",
    "InstitutionAddress": "Franklin_Blvd_1440_Eugene_Oregon_US_97403",
    "DeviceSerialNumber": "45348",
    "StationName": "AWP45348",
    "SeriesInstanceUID": "1.3.12.2.1107.5.2.19.45348.2018091609104434571015476.0.0.0",
    "StudyInstanceUID": "1.3.12.2.1107.5.2.19.45348.30000018091316524195700000031",
    "StudyID": "1",
    "PatientName": "TC002",
    "PatientID": "18.09.16-09:05:52-DST-1.3.12.2.1107.5.2.19.45348",
    "PatientBirthDate": "2000-01-01",
    "PatientSex": "O",
    "PatientWeight": 68.0389,
    "BodyPartExamined": "BRAIN",
    "PatientPosition": "HFS",
    "ProcedureStepDescription": "dsnlab_Pfeifer_TAG",
    "SoftwareVersions": "syngo_MR_E11",
    "MRAcquisitionType": "3D",
    "SeriesDescription": "mprage_p2",
    "Proto

What if you have some dicoms that AREN'T in the lcni standard directory structure? For example, these:

In [55]:
unsorted_dicoms = '/projects/lcni/jolinda/shared/TalapasClass/unsorted_dicoms/'

In [56]:
os.listdir(unsorted_dicoms)

['MR.1.3.12.2.1107.5.2.19.45348.2017121216151094687453923.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151094397553916.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151593953454273.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151069347853664.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151599158654314.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151060385153582.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151598262754309.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216162878005757311.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151593921254272.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.201712121615115383854033.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.201712121615112736554007.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151016286453131.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151594304054284.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151597169854303.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216151075106953711.dcm',
 'MR.1.3.12.2.1107.5.2.19.45348.2017121216

dicom2bids has a convenience function for sorting them.

In [57]:
dicom2bids.SortDicoms(unsorted_dicoms, class5_output.joinpath('sorted'), slurm = False)

In [58]:
!tree {class5_output.joinpath('sorted')} -d

/home/jolinda/lcni/class5out/sorted
`-- phantom_20171212_161421
    |-- Series_1_AAHScout
    |-- Series_2_AAHScout_MPR_sag
    |-- Series_3_AAHScout_MPR_cor
    |-- Series_4_AAHScout_MPR_tra
    `-- Series_5_Resting1

6 directories


Some of the clever stuff dicom2bids does to generate the dataset_description and other .json files might not work quite right (especially if your dicoms aren't from lcni), so definitely check that every time. Finally, if you want to see the commands that are called by dicom2bids.Convert without actually converting, you can do it this way (possibly useful if you are running somewhere other than talapas). You have to give it an input directory to a single subject's dicoms.

In [59]:
print(dicom2bids.GenerateCSCommand('/projects/lcni/dcm/TalapasClass/Smith/TC/TC002_20180916_090647/', bidsdir, bd))

dcm2niix -ba n -l o -o /home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/anat -f sub-TC002_ses-pre_acq-mprage_run-02_T1w  /projects/lcni/dcm/TalapasClass/Smith/TC/TC002_20180916_090647/Series_2_mprage_p2
dcm2niix -ba n -l o -o /home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/func -f sub-TC002_ses-pre_task-SVC_run-03_bold  /projects/lcni/dcm/TalapasClass/Smith/TC/TC002_20180916_090647/Series_3_SVC_1_bold_mb3_g2_2mm_te25
jq '.TaskName="SVC"' /home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/func/sub-TC002_ses-pre_task-SVC_run-03_bold.json > /tmp/sub-TC002_ses-pre_task-SVC_run-03_bold.json
mv /tmp/sub-TC002_ses-pre_task-SVC_run-03_bold.json /home/jolinda/lcni/class5out/bids/sub-TC002/ses-pre/func/sub-TC002_ses-pre_task-SVC_run-03_bold.json

